In [ ]:
from llama_index.readers.web import SimpleWebPageReader, MainContentExtractorReader, ReadabilityWebPageReader
from llama_index.core.storage.docstore import SimpleDocumentStore
import asyncio
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline, IngestionCache
from llama_index.readers.web import WholeSiteReader
from selenium import webdriver
import nest_asyncio
from llama_parse import LlamaParse
from llama_index.core.node_parser import SentenceSplitter
import os
import openai
import traceback
from llama_index.readers.web import ReadabilityWebPageReader
from tqdm.asyncio import tqdm_asyncio
from dotenv import load_dotenv


In [ ]:


load_dotenv()

In [ ]:
semaphore_count=30

In [ ]:


nest_asyncio.apply()



In [ ]:
docstore = SimpleDocumentStore()
#docstore = SimpleDocumentStore()


Below are a number of websites retrieved from google search for .gov.za site

In [ ]:
URLS = [
"http://www.cogta.gov.za",
    "http://www.dac.gov.za","http://www.dalrrd.gov.za",
    "http://www.dcs.gov.za","http://www.dhet.gov.za","http://www.dhs.gov.za","http://www.dirco.gov.za",
    "http://www.dmr.gov.za","http://www.dmv.gov.za","http://www.doc.gov.za","http://www.dod.mil.za",
    #"http://www.doh.gov.za",
    "http://www.dpe.gov.za","http://www.dpsa.gov.za","http://www.dsbd.gov.za",
    "http://www.dsd.gov.za","http://www.dst.gov.za","http://www.dta.gov.za","http://www.dtps.gov.za",
    "http://www.dwa.gov.za","http://www.economic.gov.za","http://www.education.gov.za",
    #"http://www.energy.gov.za",
    "http://www.environment.gov.za","http://www.gcis.gov.za","http://www.gov.za",
    "http://www.home-affairs.gov.za","http://www.ipid.gov.za","http://www.judiciary.org.za","http://www.justice.gov.za",
    "http://www.labour.gov.za","http://www.policesecretariat.gov.za","http://www.psc.gov.za","http://www.publicworks.gov.za",
    "http://www.ruraldevelopment.gov.za","http://www.saps.gov.za","http://www.sars.gov.za","http://www.srsa.gov.za",
    "http://www.ssa.gov.za","http://www.statssa.gov.za","http://www.thedti.gov.za","http://www.thensg.gov.za",
    "http://www.thepresidency-dpme.gov.za","http://www.thepresidency.gov.za","http://www.tourism.gov.za",
    "http://www.transport.gov.za","http://www.treasury.gov.za","http://www.wcpd.gov.za",
    "https://old.dalrrd.gov.za","https://www.gcis.gov.za",
    "https://www.gov.za/issues/national-development-plan-2030",
    "http://www.cogta.gov.za",
    "http://www.dac.gov.za","http://www.dalrrd.gov.za","http://www.dcs.gov.za","http://www.dhet.gov.za",
    "http://www.dhs.gov.za","http://www.dirco.gov.za","http://www.dmr.gov.za","http://www.dmv.gov.za","http://www.doc.gov.za",
    "http://www.dod.mil.za","http://www.doh.gov.za",
    "http://www.dpe.gov.za","http://www.dpsa.gov.za","http://www.dsbd.gov.za",
    "http://www.dsd.gov.za","http://www.dst.gov.za",
    "http://www.dta.gov.za","http://www.dtps.gov.za","http://www.dwa.gov.za",
    "http://www.economic.gov.za","http://www.education.gov.za",
    #"http://www.energy.gov.za",
    "http://www.environment.gov.za",
    "http://www.gcis.gov.za","http://www.gov.za",
    "http://www.home-affairs.gov.za","http://www.ipid.gov.za",
    "http://www.judiciary.org.za","http://www.justice.gov.za",
    "http://www.labour.gov.za","http://www.policesecretariat.gov.za",
    "http://www.psc.gov.za","http://www.publicworks.gov.za",
    "http://www.ruraldevelopment.gov.za","http://www.saps.gov.za",
    "http://www.sars.gov.za",
    "http://www.ssa.gov.za","http://www.statssa.gov.za",
    "http://www.thedti.gov.za","http://www.thensg.gov.za",
    #"http://www.thepresidency-dpme.gov.za",
    "http://www.thepresidency.gov.za",
    "http://www.tourism.gov.za","http://www.transport.gov.za","http://www.treasury.gov.za","http://www.wcpd.gov.za",
    "https://old.dalrrd.gov.za","https://www.gcis.gov.za","https://www.gov.za",
    "https://www.sars.gov.za",
    "https://www.health.gov.za",
    "https://www.capetown.gov.za",
    "https://www.transport.gov.za",
    "https://www.eservices.gov.za",
    "https://www.labour.gov.za",
    "https://www.thedtic.gov.za",
    "https://www.sassa.gov.za",
    "https://www.ssa.gov.za",
    "https://www.treasury.gov.za",
    "https://www.statssa.gov.za",
    "https://www.judiciary.org.za",
    "https://www.ipid.gov.za",
    "https://www.dcs.gov.za", "https://www.cogta.gov.za", "https://www.dcdt.gov.za",
    "https://www.transport.gov.za", "https://www.dws.gov.za", "https://www.dwypd.gov.za","https://www.thensg.gov.za",
    "https://www.cipc.co.za", "https://www.npo.gov.za", "https://www.westerncape.gov.za", "https://ecert.co.za",
    "https://online.natis.gov.za", "https://www.gauteng.gov.za", "https://www.etenders.gov.za", "https://westerncape.gov.za",
    "https://www.thepresidency.gov.za", "https://secure.csd.gov.za", "https://www.nyda.gov.za", "https://www.dffe.gov.za",
    "https://twk.gov.za","https://www.npa.gov.za", "https://www.gems.gov.za", "https://www.dsac.gov.za",
    "https://www.fic.gov.za",
    "https://www.durban.gov.za", "https://www.gardenroute.gov.za","https://www.capetown.gov.za", "https://www.dsti.gov.za",
    "http://www.dmv.gov.za", "https://www.gtac.gov.za", "http://www.epwp.gov.za","https://www.chartgov.co.za",
    "https://www.pic.gov.za", "https://www.capewinelands.gov.za","https://www.gpwonline.co.za",
    "https://www.dhs.gov.za", "https://www.tshwane.gov.za", "https://www.ekurhuleni.gov.za",
    "https://www.cipc.co.za",
    "https://www.gov.za",
    'http://www.kznhealth.gov.za', 'https://www.justice.gov.za', 'https://www.kzncogta.gov.za', 'https://www.kznhealth.gov.za', 'https://www.mbombela.gov.za', 'https://www.thedtic.gov.za', 'http://www.ecdsd.gov.za', 'http://www.kznhealth.gov.za', 'https://dirco.gov.za', 'https://www.dpme.gov.za', 'https://www.emalahlenilm.gov.za', 'https://www.kzndsd.gov.za', 'https://www.laingsburg.gov.za', 'https://www.sars.gov.za', 'http://www.kznhealth.gov.za', 'https://www.blouberg.gov.za', 'https://www.dffe.gov.za', 'https://www.kznhealth.gov.za', 'https://www.mbombela.gov.za', 'https://www.raymondmhlaba.gov.za', 'https://www.saps.gov.za', 'https://www.sars.gov.za', 'http://www.kznhealth.gov.za', 'https://dirco.gov.za', 'https://www.blouberg.gov.za', 'https://www.kznhealth.gov.za', 'https://www.mbombela.gov.za', 'https://www.mpg.gov.za', 'https://www.nationalarchives.gov.za', 'https://www.saps.gov.za', 'https://www.sars.gov.za', 'http://www.kznhealth.gov.za', 'https://dirco.gov.za', 'https://www.blouberg.gov.za', 'https://www.kznhealth.gov.za', 'https://www.mbombela.gov.za', 'https://www.mpg.gov.za', 'https://www.nationalarchives.gov.za', 'https://www.saps.gov.za', 'https://www.sars.gov.za', 'https://dirco.gov.za', 'https://www.amajuba.gov.za', 'https://www.capetown.gov.za', 'https://www.capewinelands.gov.za', 'https://www.dmre.gov.za', 'https://www.emfuleni.gov.za', 'https://www.health.gov.za', 'https://www.kznhealth.gov.za', 'https://www.mosselbay.gov.za', 'http://www.kznhealth.gov.za', 'https://bnlm.gov.za', 'https://dirco.gov.za', 'https://vulekamali.gov.za', 'https://www.kznhealth.gov.za', 'https://www.parliament.gov.za', 'https://www.sars.gov.za', 'https://www.srvm.gov.za', 'http://www.kznhealth.gov.za', 'https://bnlm.gov.za', 'https://dirco.gov.za', 'https://vulekamali.gov.za', 'https://www.kznhealth.gov.za', 'https://www.parliament.gov.za', 'https://www.sars.gov.za', 'https://www.srvm.gov.za', 'http://www.kznhealth.gov.za', 'http://www.publicworks.gov.za', 'https://www.dsac.gov.za', 'https://www.ekurhuleni.gov.za', 'https://www.kznhealth.gov.za', 'https://www.lwr.gov.za', 'https://www.rustenburg.gov.za', 'https://bvm.gov.za', 'https://dirco.gov.za', 'https://durbanmarkets.durban.gov.za', 'https://www.chrishanidm.gov.za', 'https://www.ekurhuleni.gov.za', 'https://www.gov.za', 'https://www.ldoh.gov.za', 'http://www.kznhealth.gov.za', 'https://dirco.gov.za', 'https://www.dhs.gov.za', 'https://www.dsac.gov.za', 'https://www.gov.za', 'https://www.health.gov.za', 'https://www.kznhealth.gov.za', 'https://www.nwpg.gov.za', 'https://dirco.gov.za', 'https://sbm.gov.za', 'https://vulekamali.gov.za', 'https://www.dmre.gov.za', 'https://www.ecdpw.gov.za', 'https://www.gardenroute.gov.za', 'https://www.health.gov.za', 'https://www.madibeng.gov.za', 'https://www.sars.gov.za', 'https://www.thensg.gov.za', 'https://dirco.gov.za', 'https://sbm.gov.za', 'https://vulekamali.gov.za', 'https://www.dmre.gov.za', 'https://www.ecdpw.gov.za', 'https://www.gardenroute.gov.za', 'https://www.health.gov.za', 'https://www.madibeng.gov.za', 'https://www.sars.gov.za', 'https://www.thensg.gov.za', 'https://dirco.gov.za', 'https://vulekamali.gov.za', 'https://www.dmre.gov.za', 'https://www.ecdpw.gov.za', 'https://www.gardenroute.gov.za', 'https://www.health.gov.za', 'https://www.matatiele.gov.za', 'https://www.thensg.gov.za', 'https://dirco.gov.za', 'https://www.drdar.gov.za', 'https://www.gpl.gov.za', 'https://www.justice.gov.za', 'https://www.sars.gov.za', 'https://www.statssa.gov.za', 'https://www.westerncape.gov.za', 'https://www.statssa.gov.za', 'https://www.tshwane.gov.za', 'https://www.statssa.gov.za', 'https://www.tshwane.gov.za', 'https://www.statssa.gov.za', 'https://dirco.gov.za', 'https://eceducation.gov.za', 'https://wcedonline.westerncape.gov.za', 'https://www.emalahleni.gov.za', 'https://www.gov.za', 'https://www.nelsonmandelabay.gov.za', 'https://www.statssa.gov.za', 'https://www.transport.gov.za', 'http://socdev.ncpg.gov.za', 'http://www.kznonline.gov.za', 'https://dirco.gov.za', 'https://www.dha.gov.za', 'https://www.fic.gov.za', 'https://www.gpl.gov.za', 'https://www.langeberg.gov.za', 'https://www.nelsonmandelabay.gov.za', 'https://www.rnm.gov.za', 'https://www.transport.gov.za', 'https://bnlm.gov.za', 'https://dirco.gov.za', 'https://ndlambe.gov.za', 'https://ortambodm.gov.za', 'https://www.dhet.gov.za', 'https://www.gov.za', 'https://www.hessequa.gov.za', 'https://www.sars.gov.za', 'https://www.westerncape.gov.za', 'https://brics2023.gov.za', 'https://dirco.gov.za', 'https://ecprov.gov.za', 'https://feziledabi.gov.za', 'https://ksd.gov.za', 'https://musina.gov.za', 'https://sbm.gov.za', 'https://www.dmre.gov.za', 'https://www.gtac.gov.za', 'https://www.kouga.gov.za', 'https://unemployededucators.gauteng.gov.za', 'https://www.dhet.gov.za', 'https://www.sars.gov.za', 'https://www.statssa.gov.za', 'https://www.transport.gov.za', 'https://www.tshwane.gov.za', 'https://www.justice.gov.za', 'https://www.sars.gov.za', 'https://www.statssa.gov.za', 'https://www.transport.gov.za', 'https://www.tshwane.gov.za', 'https://www.dha.gov.za', 'https://www.dhet.gov.za', 'https://www.justice.gov.za', 'https://www.sars.gov.za', 'https://www.statssa.gov.za', 'https://www.transport.gov.za', 'https://www.tshwane.gov.za', 'https://ipwis.westerncape.gov.za', 'https://www.dhet.gov.za', 'https://www.ecdhs.gov.za', 'https://www.education.gov.za', 'https://www.moroka.gov.za', 'https://www.sars.gov.za', 'https://www.statssa.gov.za', 'https://www.tshwane.gov.za', 'http://www.kznhealth.gov.za', 'https://admissions.westerncape.gov.za', 'https://dirco.gov.za', 'https://www.capewinelands.gov.za', 'https://www.education.gov.za', 'https://www.ekurhuleni.gov.za', 'https://www.statssa.gov.za', 'http://www.kznhealth.gov.za', 'https://bvm.gov.za', 'https://dirco.gov.za', 'https://www.polokwane.gov.za', 'http://www.kznhealth.gov.za', 'https://www.justice.gov.za', 'https://www.kzncogta.gov.za', 'https://www.kznhealth.gov.za', 'https://www.mbombela.gov.za', 'https://www.thedtic.gov.za'
]

In [ ]:
# Removes duplicates
URLS = list(set(URLS))

In [ ]:
print(len(URLS))

In [ ]:
links = []

In [ ]:
js_code = [
                "window.scrollTo(0, document.body.scrollHeight);",
                "document.querySelector('.load-more')?.click();",
                "document.querySelector('#consent-button')?.click();"
]



The Below crawl4AI is how I spider/crawl the government websites for links. It looks for all 
<a href=""> tags

In [ ]:
import asyncio
from crawl4ai import AsyncWebCrawler

async def scrape_links_async(sem, url):
    async with sem:
        async with AsyncWebCrawler(verbose=False) as crawler:
            result = await crawler.arun(
                url=url + "/",
                    exclude_external_links=True,         # Remove external links
                    exclude_social_media_links=True ,     # Remove social media links
                #remove_overlay_elements=True,
                magic=True,
                simulate_user=True,
                remove_overlay_elements=True,
                page_timeout=90000,              # Page load timeout (ms)
                delay_before_return_html=5.0,
                js_code=js_code
        
            )
    
            #print(result.markdown[:100])
            _links = [] + result.links.get("internal", []) + result.links.get("external",[])
            _links = [ x.get("href", "") for x in _links]
            _links = [ x for x in _links if "http://" in x or "https://" in x]
            _links = [ x for x in _links if "gov.za" in x or ".co.za" in x]
            #_links = [ x for x in _links if ".gov.za" in x or ".co.za" in x]
            #print(_links)
            return _links

#asyncio.run(main())

In [ ]:
async def run_async_scrape(links):
    sem = asyncio.Semaphore(semaphore_count)
    print("Setting up tasks")
    tasks = [asyncio.create_task(scrape_links_async(sem, x.replace('//', '/').replace(':/', '://'))) for x in URLS ]
    print("await gather")
    await tqdm_asyncio.gather(*tasks)
    print(len(tasks))
    for x in tasks:
        links = links + x.result()
    links = [ x.replace('//', '/').replace(':/', '://') for x in links]
    return links



In [ ]:
# First round of spidering
links = asyncio.run(run_async_scrape(links))


In [ ]:
URLS = list(set(links))

In [ ]:
print(len(URLS))

In [28]:
# Second round of spidering, just in case
links = list(set(asyncio.run(run_async_scrape(URLS))))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6637/6637 [30:31<00:00,  3.62it/s]

[ERROR] 🚫 arun(): Failed to crawl https://wcedonline.westerncape.gov.za/contact/districts/, error: [ERROR] 🚫 crawl(): Failed to crawl https://wcedonline.westerncape.gov.za/contact/districts/: Timeout 30000ms exceeded.
6637


In [30]:
# This function splits document URLs from HTML URLs
# Web Scraper doesn't handle documents at the moment
def clean_urls(urls):
    doc_urls = []
    # Remove PDFs
    doc_urls = doc_urls + [x for x in urls if ".pdf" in x]
    doc_urls = doc_urls +[x for x in urls if ".xlsx" in x]
    doc_urls = doc_urls + [x for x in urls if ".xls" in x]
    doc_urls = doc_urls + [x for x in urls if ".docx" in x]
    doc_urls = doc_urls +[x for x in urls if ".doc" in x]
    
    urls = [x for x in urls if ".pdf" not in x]
    urls = [x for x in urls if ".xlsx" not in x]
    urls = [x for x in urls if ".xls" not in x]
    urls = [x for x in urls if ".docx" not in x]
    urls = [x for x in urls if ".doc" not in x]
    urls = [x for x in urls if ".jpg" not in x]
    urls = [x for x in urls if ".zip" not in x]
    #print(urls)
    #return [x.metadata['URL'] for x in urls], [x.metadata['URL'] for x in doc_urls]
    return urls, doc_urls

In [32]:
new_urls, doc_urls = clean_urls(URLS)

In [34]:
print(len(new_urls))

5700


In [35]:
print(len(doc_urls))

848


In [40]:
content = []
maincontent_web_docs = []
counter = 0

In [41]:
# Scraper JS commands for Dynamic Pages
js_commands = [
    "window.scrollTo(0, document.body.scrollHeight);",
    "document.querySelector('.load-more')?.click();"
    #"document.querySelector('.load-more').click();",
    "document.querySelector('#consent-button')?.click();"
]

Below is the web scraper.
This code is async so very fast, and enables webpage to markdown conversion
Can be attached to an LLM to scrape as well

In [49]:
import asyncio
from crawl4ai import AsyncWebCrawler

async def crawl(sem, url):
    async with sem:
        async with AsyncWebCrawler(verbose=False) as crawler:
            result = await crawler.arun(
                url=url,
                js_code=js_code,
                word_count_threshold=2,        # Minimum words per content block
                exclude_external_links=True,    # Remove external links
                remove_overlay_elements=True,   # Remove popups/modals
                process_iframes=True,          # Process iframe content
                #excluded_tags=['form', 'nav'], # Exclude specific tags
            )
    
            #print(result.markdown[:100])
            return result
            #return result

#asyncio.run(main())

In [50]:

content = []

In [58]:
# Executes the crawler async with sempahores to limit the web scrape volume
async def run_async_crawl(content):
    sem = asyncio.Semaphore(semaphore_count)
    print("Setting up tasks")
    tasks = [asyncio.create_task(crawl(sem,x)) for x in new_urls[:5] ]
    print("await gather")
    await tqdm_asyncio.gather(*tasks)
    print(len(tasks))
    for x in tasks:
        content.append(x.result())
    #content = [ x for x in content]
    return content



In [ ]:
# Scrape all the links
content = asyncio.run(run_async_crawl(content))

[LOG] 🚀 Content extracted for https://twk.gov.za/category/documents/vacancies/employment-application-form/, success: True, time taken: 0.18 seconds
[LOG] 🚀 Extraction done for https://twk.gov.za/category/documents/vacancies/employment-application-form/, time taken: 0.18 seconds.
[LOG] 🚀 Content extracted for http://www.dalrrd.gov.za/index.php/about-us/vision-and-mission, success: True, time taken: 0.11 seconds
[LOG] 🚀 Extraction done for http://www.dalrrd.gov.za/index.php/about-us/vision-and-mission, time taken: 0.11 seconds.
[LOG] 🚀 Content extracted for https://www.durban.gov.za/pages/residents/general-valuation, success: True, time taken: 0.06 seconds
[LOG] 🚀 Extraction done for https://www.durban.gov.za/pages/residents/general-valuation, time taken: 0.06 seconds.
[LOG] 🚀 Content extracted for https://www.kouga.gov.za/document-library, success: True, time taken: 4.04 seconds
[LOG] 🚀 Extraction done for https://www.kouga.gov.za/document-library, time taken: 4.05 seconds.
[ERROR] 🚫 ar

In [55]:
docs = []

In [56]:
# This code converts from the makrdown format of crawl4ai into the Document object of llamaindex..
# Lamaindex is our indexing and RAG framework, so need the data in the correct format.
# Could be achieved by saving markdown or JSON and using DirectoryReader as well.
for x in content:
    if isinstance(x, dict):
        _doc = Document(text=x.fit_markdown)
        #print(x.__dict__)
        _doc.metadata['url'] = x.__dict__.get('url')
        _doc.metadata['title'] = x.__dict__.get('metadata',{}).get('title')
        _doc.metadata['keywords'] = x.__dict__.get('metadata',{}).get('keywords')
        _doc.metadata['last-modified'] = x.response_headers.get('last-modified')
        _doc.metadata['date'] = x.response_headers.get('date')
        _doc.metadata['content-type'] = x.response_headers.get('content-type')
        docs.append(_doc)

In [ ]:
# Old slower Scraping code
# for x in new_urls:
#     try:
#         _ = ReadabilityWebPageReader(wait_until='domcontentloaded').load_data(url=x.metadata['URL'])
#         #_ = MainContentExtractorReader().load_data(urls=[x.metadata['URL']])
#         _[0].metadata = x.metadata
#         _[0].metadata['original_text'] = x.text
#         maincontent_web_docs = maincontent_web_docs + _ 
#     except Exception as e:
#         print(f"Failed to load: {x.metadata['URL']}")
#         print(e)
#         try:
#             _ = MainContentExtractorReader().load_data(urls=[x.metadata['URL']])
#             _[0].metadata = x.metadata
#             _[0].metadata['original_text'] = x.text
#             maincontent_web_docs = maincontent_web_docs + _ 
#         except Exception as e:
#             print(e)
# # for url in new_urls:
# #     maincontent_web_docs = maincontent_web_docs + 
    

In [57]:
print(docs[2])

IndexError: list index out of range

In [ ]:
print(len(docs)

In [ ]:
#docs = docs + web_docs

In [ ]:
# Removes empty text or errors
docs = [x for x in docs if x.text != ""]
docs = [x for x in docs if "error" not in x.text.lower()]

In [ ]:
for x in docs:
    if "Forbidden nginx" in x.text[:40]:
    
        print(x.text[:400])
        print(x.metadata)


In [ ]:
print(len(docs))


In [ ]:
docstore = SimpleDocumentStore()


In [ ]:
docstore.add_documents(docs)

In [ ]:

docstore.persist()

In [ ]:
# import
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.elasticsearch import ElasticsearchStore
from llama_index.core import StorageContext

In [ ]:
elastic_cloud_id = os.getenv("es_cloud_id")
elastic_user = os.getenv("es_user")
elastic_password = os.getenv("es_password")

Below connects to Elasticsearch vector Datbase


In [ ]:
from llama_index.vector_stores.elasticsearch import ElasticsearchStore

vector_store = ElasticsearchStore(
    index_name="gov_web_scrape_last",
    es_cloud_id=elastic_cloud_id, # found within the deployment page
    es_user=elastic_user,
    es_password=elastic_password # provided when creating deployment. Alternatively can reset password.
)


Below is using Ollama (opensource model running app) and nomic-embed-text a opensource embedding model
You could replace Ollama and nomic-embed-text with OpenAI embedding

In [ ]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings
Settings.embed_model = OllamaEmbedding(
        model_name="nomic-embed-text",
        embed_batch_size=100,
        ollama_additional_kwargs={"num_ctx": 8192},
)


In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    new_docs, storage_context=storage_context, embed_model=Settings.embed_model, chunk_size=2048
)

In [ ]:


from llama_index.llms.ollama import Ollama


In [ ]:
Below is using Ollama (opensource model running app) and mistral-7b a opensource french LLM model
You could replace Ollama and mistral with OpenAI LLM like gpt4

In [ ]:
llm = Ollama(model="mistral:latest", request_timeout=120.0)

Below we setup the query engine
The query engine in llamaindex allows querying documents and then sending the documents to a LLM

In [ ]:
query_engine = index.as_query_engine(llm=llm, embed_model=Settings.embed_model, similarity_top_k=10)


In [ ]:
%%time
response = query_engine.query("Where in Cape Town, Western Cape can I visit a SARS branch?",  )
print(response)
print(response.metadata)